### Import Necessary Libraries

First, we import the necessary libraries for our optimization model. These include pandas and numpy for data handling and manipulation, os for file operations, math for mathematical functions, networkx for graph-related computations, and gurobipy for optimization.

In [ ]:
import pandas as pd
import numpy as np
import os
import math
import networkx as nx
from gurobipy import Model, GRB, quicksum

### Generation of traffic flow data $R_{k.t.n}$


We generate traffic flow data using data from the [National Highways of the UK](https://webtris.highwaysengland.co.uk/). This involves loading data from CSV files and aggregating traffic flow information for each node.

In [ ]:
# Load the Excel file
highway_node = pd.read_csv("data/Node.csv", header=None, delimiter=',')

# Initialize R as a 3D numpy array
R = np.zeros((14, 24, 30))

# Function to clean and extract data from the CSV files
def process_csv(file_path):
    data = pd.read_csv(file_path, skiprows=3, header=0, delimiter=',')
    data = data.astype(str).replace('nan', '')
    return data.values

# Iterate over each node
for k in range(1, 15):
    A = highway_node[highway_node.iloc[:, 6] == k].index.tolist()
    for i in A:
        file_path = os.path.join('data', highway_node.iloc[i, 5])
        Data = process_csv(file_path)
        for n in range(30):
            for t in range(6, 24):
                try:
                    start_row = 96 * n + 4 * (t-1) + 4
                    end_row = start_row + 4
                    R[k-1, t-1, n] += np.sum(pd.to_numeric(Data[start_row:end_row, 4]))
                except Exception as e:
                    print(f"Error processing data for node {k}, day {n}, time {t}: {e}")
                    continue

R = np.nan_to_num(R,nan=0)

print("R_k.t.n generated successfully.")

### Generation of the traffic network $G$

We define the traffic network using a graph $G$, where nodes represent intersections and edges represent roads between them. The edges are weighted by the distance between nodes.

In [ ]:
# Define the edges (assuming node indexing starts from 1)
edges = [
    (1, 2, 1),
    (2, 3, 3.5),
    (3, 4, 14),
    (4, 5, 3.8),
    (5, 6, 17.5),
    (6, 7, 14.5),
    (7, 8, 4.1),
    (8, 9, 6.9),
    (9, 10, 15.5),
    (10, 11, 3.3),
    (11, 12, 10.7),
    (12, 13, 13),
    (13, 14, 2.7),
    (14, 1, 7.9)
]

# Create a graph
G = nx.Graph()

# Add edges to the graph
for u, v, w in edges:
    G.add_edge(u, v, weight=w)

# Ensure node labels start from 0
G = nx.convert_node_labels_to_integers(G, first_label=0)

print("Graph G generated successfully.")

### Generation of the distance matrix $d_{i.j}$, the transportation time matrix $T_{i.j}$, and the path indicator parameter $a_{i.j.k}$

We calculate the shortest paths and distances between all pairs of nodes using Dijkstra's algorithm. The distance matrix $d_{i.j}$ stores the shortest distance between nodes $i$ and $j$, the transportation time matrix $T_{i.j}$ is derived from the distance matrix, and the path indicator parameter $a_{i.j.k}$ indicates whether node $k$ lies on the shortest path from $i$ to $j$.

In [ ]:
# Initialize the distance matrix
num_nodes = G.number_of_nodes()
d = np.zeros((num_nodes, num_nodes))

# Initialize the indicator matrix
a = np.zeros((num_nodes, num_nodes, num_nodes), dtype=int)

# Compute shortest paths and distances using Dijkstra's algorithm
for i in range(num_nodes):
    lengths, paths = nx.single_source_dijkstra(G, i)
    for j in range(num_nodes):
        if i != j:
            d[i, j] = lengths[j]
            path = paths[j]
            for k in range(1, len(path) - 1):
                intermediate = path[k]
                a[i, j, intermediate] = 1

Tij = d / 100

print("d_i.j, T_i.j and a_i.j.k generated successfully.")

### Definition of other parameters

We define various parameters required for the optimization model. These include node sets, costs, time-related parameters, and other constants.

In [ ]:
# Parameters
N = [i for i in range(1, 15)]
C1, C2, C3 = 100, 50, 100
fc = lambda c: 200 + 50 * c
gc = lambda c: 300 + 300 * c
hc = lambda c: 200 + 30 * c
theta = 10
r = 30
TC = 2
TB = 1/6
Ttol = 1/2
epsilon = 0.8
w = 0.0365
OB, OE = 6, 23
observed_days = 30

### Calculation of Value at Risk $V_{k.t}$

We calculate the Value at Risk (VaR) for each node $k$ and time $t$. This involves summing the traffic flow data over specific axes and extracting subsets to compute the VaR.

In [ ]:
# Calculate Rn as the sum over the first two axes
R_n = np.sum(R, axis=(0, 1))

# Position refers to the index of the bound of Value-at-Risk.
position = int(np.ceil(observed_days * epsilon)) - 1

# Get the index of the value at the position after sorting
n = np.argpartition(R_n, position)[position]

# Extract the subset for the specified index and calculate RVaR
subset = R[:, :, n]
V_kt = np.ceil(0.005 * subset)

### Optimization Model

We define and solve the optimization model for EV infrastructure planning. The model includes decision variables for placement of facilities, flows between nodes, and other relevant variables. The objective function minimizes the total cost, and various constraints ensure the feasibility of the solution.

In [ ]:
# Create a new model
model = Model("EV Infrastructure Planning")

# Decision variables
A = model.addVars(N, range(1, C1+1), vtype=GRB.BINARY, name="A")
B = model.addVars(N, range(1, C2+1), vtype=GRB.BINARY, name="B")
O = model.addVars(N, range(1, C3+1), vtype=GRB.BINARY, name="O")
L = model.addVars(N, vtype=GRB.BINARY, name="L")
H = model.addVars(N, range(1, 25), vtype=GRB.INTEGER, name="H")
C = model.addVars(N, range(1, 25), vtype=GRB.INTEGER, name="C")
M = model.addVars(N, range(1, 25), vtype=GRB.INTEGER, name="M")
N_var = model.addVars(N, range(1, 25), vtype=GRB.INTEGER, name="N")
F = model.addVars(N, range(1, 25), vtype=GRB.INTEGER, name="F")
E = model.addVars(N, range(1, 25), vtype=GRB.INTEGER, name="E")
D = model.addVars(N, N, range(1, 25), vtype=GRB.INTEGER, name="D")
G = model.addVars(N, N, range(1, 25), vtype=GRB.INTEGER, name="G")
I = model.addVars(N, vtype=GRB.INTEGER, name="I")

# Objective function
model.setObjective(
    quicksum(fc(c) * A[k, c] for k in N for c in range(1, C1+1)) +
    quicksum(gc(c) * B[k, c] for k in N for c in range(1, C2+1)) +
    quicksum(hc(c) * O[k, c] for k in N for c in range(1, C3+1)) +
    quicksum(w * d[i-1, j-1] * D[i, j, t] for i in N for j in N if i != j for t in range(OB, OE+1)) +
    quicksum(w * d[i-1, j-1] * G[i, j, t] for i in N for j in N if i != j for t in range(OB, OE+1)) +
    quicksum(theta * I[k] for k in N),
    GRB.MINIMIZE
)

# Constraints
model.addConstrs((quicksum(A[k, c] for c in range(1, C1+1)) <= 1 for k in N), name="C2")
model.addConstrs((quicksum(B[k, c] for c in range(1, C2+1)) <= 1 for k in N), name="C3")
model.addConstrs((quicksum(O[k, c] for c in range(1, C3+1)) <= 1 for k in N), name="C4")
model.addConstrs((L[k] <= quicksum(A[k, c] for c in range(1, C1+1)) + quicksum(B[k, c] for c in range(1, C2+1)) for k in N), name="C5")
model.addConstrs((L[k] >= 0.5 * (quicksum(A[k, c] for c in range(1, C1+1)) + quicksum(B[k, c] for c in range(1, C2+1))) for k in N), name="C6")
model.addConstrs((float(d[i-1, j-1]) <= r * (1 + quicksum(L[k] * int(a[i-1, j-1, k-1]) for k in N)) for i in N for j in N if i != j), name="C7")
model.addConstrs((F[k, OB] == I[k] for k in N), name="C8")
model.addConstrs((F[k, OE] == I[k] for k in N), name="C9")
model.addConstrs((E[k, OB] == 0 for k in N), name="C10")

model.addConstrs((
    F[k, t] == F[k, t-1] - H[k, t-1] + (M[k, t-TC] if t-TC >= 1 else 0) + (N_var[k, t-TC] if t-TC >= 1 else 0) -
    quicksum(D[k, j, t-1] for j in N if k != j) +
    quicksum(D[j, k, t-int(math.ceil(Tij[j-1, k-1]))] if t-int(math.ceil(Tij[j-1, k-1])) >= 1 else 0 for j in N if k != j)
    for k in N for t in range(OB, OE+1)
), name="C11")

model.addConstrs((
    E[k, t] == E[k, t-1] + H[k, t-1] - M[k, t-1] - N_var[k, t-1] -
    quicksum(G[k, j, t-1] for j in N if k != j) +
    quicksum(G[j, k, t-int(math.ceil(Tij[j-1, k-1]))] if t-int(math.ceil(Tij[j-1, k-1])) >= 1 else 0 for j in N if k != j)
    for k in N for t in range(OB, OE+1)
), name="C12")

model.addConstrs((M[k, t] + N_var[k, t] <= E[k, t] for k in N for t in range(OB, OE+1)), name="C13")
model.addConstrs((quicksum(M[k, l] for l in range(max(1, t-TC+1), t+1)) <= quicksum(c * A[k, c] for c in range(1, C1+1)) for k in N for t in range(OB, OE+1)), name="C14")
model.addConstrs((quicksum(N_var[k, l] for l in range(max(1, t-TC+1), t+1)) <= quicksum(c * O[k, c] for c in range(1, C3+1)) for k in N for t in range(OB, OE+1)), name="C15")
model.addConstrs((H[k, t] <= (1/TB) * quicksum(c * B[k, c] for c in range(1, C2+1)) for k in N for t in range(OB, OE+1)), name="C16")

model.addConstrs((quicksum(D[k, j, t] for j in N if k != j) <= F[k, t] - H[k, t] for k in N for t in range(OB, OE+1)), name="C17")
model.addConstrs((quicksum(G[k, j, t] for j in N if k != j) <= E[k, t] + H[k, t] for k in N for t in range(OB, OE+1)), name="C18")

model.addConstrs((
    quicksum(F[k, t] + E[k, t] + quicksum(M[k, l] + N_var[k, l] for l in range(t-TC+1, t+1)) +
    quicksum(D[k, j, t-int(math.ceil(Tij[k-1, j-1]))] + G[k, j, t-int(math.ceil(Tij[k-1, j-1]))] if t-int(math.ceil(Tij[k-1, j-1])) >= 1 else 0 for j in N if k != j)
    for k in N) >= quicksum(I[k] for k in N)
    for t in range(OB, OE+1)
), name="C19")

model.addConstrs((C[k, t] <= (quicksum(c * A[k, c] for c in range(1, C1+1)) - quicksum(M[k, l] for l in range(max(1, t-TC+1), t+1)) - quicksum(C[k, l] for l in range(max(1, t-TC), t)))
                 for k in N for t in range(OB, OE+1)), name="C20")

model.addConstrs((C[k, t] + H[k, t] - float(1/Ttol) * L[k] >= float(V_kt[k-1,t-1]) for k in N for t in range(OB, OE+1)), name="C24")

# Set variables to 0 outside of operational time
for k in N:
    for t in range(1, 25):
        if t < OB or t > OE:
            model.addConstr(H[k, t] == 0)
            model.addConstr(C[k, t] == 0)
            model.addConstr(M[k, t] == 0)
            model.addConstr(N_var[k, t] == 0)
            model.addConstr(quicksum(D[k, j, t] for j in N if k != j) == 0)
            model.addConstr(quicksum(G[k, j, t] for j in N if k != j) == 0)

model.Params.TimeLimit = 100  # Maximum running time in seconds
model.Params.MIPGap = 0.01  # Relative optimality gap

model.optimize()

if model.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    for v in model.getVars():
        if v.x > 0:
            print(f"{v.varName}: {v.x}")
else:
    print("No optimal solution found.")
    if model.status == GRB.INFEASIBLE:
        print("Model is infeasible. Computing IIS...")
        model.computeIIS()
        model.write("model.ilp")
        print("IIS written to 'model.ilp'")

        # Read and print the conflicting constraints
        with open("model.ilp", "r") as file:
            for line in file:
                print(line.strip())

Here's a guide on encountering no bool value error information: [Constraint has no bool value (are you trying "lb <= expr <= ub"?)](https://support.gurobi.com/hc/en-us/articles/360039628832-Constraint-has-no-bool-value-are-you-trying-lb-expr-ub)